In [ ]:
import numpy as np
import pandas as pd
import pydeck as pdk
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from tqdm import tqdm
from geopy.distance import geodesic
import xarray as xr
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from astral.sun import sun
from astral.location import Observer
from datetime import datetime, timedelta
from scipy.spatial import cKDTree
import pytz
import random
import math
import pickle
from pyEDM import *
import json
from pyproj import Proj
from pyhdf.SD import SD, SDC
from scipy.spatial import cKDTree

## Load Data

In [ ]:
# Load Integrated NPP Data from CalCOFI Bottle Survey
npp = pd.read_csv('../data/CalCOFI_Integrated_NPP.csv', index_col=0)
npp = npp[['Date', 'Latitude', 'Longitude', 'Integrated_NPP']]
npp['Date'] = pd.to_datetime(npp['Date'])
npp = npp.sort_values(by='Date').reset_index(drop=True)
npp = npp[npp.Date >= '1997-09-04'].reset_index(drop=True)

## Define Functions

In [ ]:
def calculate_grid_window(res_km, target_area_km2=729, valid_fraction=1/3):
    """
    Calculate the size of an n x n satellite pixel grid that best approximates a target spatial area, 
    and determine the minimum number of valid pixels required within that grid based on a valid fraction.
    """
    
    # Calculate the ideal grid size (in pixels) to match the target spatial area
    ideal_grid_size = math.sqrt(target_area_km2 / (res_km ** 2))
    
    # Round ideal size to nearest integer to get base grid size
    base_grid_size = round(ideal_grid_size)
    
    # Find nearest odd integer grid sizes: one just below or equal, and one just above
    odd_floor = base_grid_size if base_grid_size % 2 == 1 else base_grid_size - 1
    odd_ceil = odd_floor + 2  # Next odd number above odd_floor
    
    # Compute the actual spatial areas these two grid sizes represent
    area_floor = (odd_floor * res_km) ** 2
    area_ceil = (odd_ceil * res_km) ** 2
    
    # Choose the odd grid size with the area closest to the target area
    if abs(area_floor - target_area_km2) <= abs(area_ceil - target_area_km2):
        n = odd_floor
    else:
        n = odd_ceil
    
    # Calculate total number of pixels in the grid window
    total_pixels = n ** 2
    
    # Calculate the minimum number of valid pixels required, rounding to nearest integer
    min_valid_pixels = int(valid_fraction * total_pixels + 0.5)
    
    return n, min_valid_pixels

In [ ]:
def load_chl_day(filename):
    """Loads HDF, applies scaling, and returns data + projection info."""
    hdf = SD(f'../data/chl_data/{filename}', SDC.READ)
    
    # Identify the primary dataset (usually the first one)
    sds_name = list(hdf.datasets().keys())[0]
    sds = hdf.select(sds_name)
    data = sds[:].astype(np.float32)
    
    # Get scaling attributes
    attrs = sds.attributes()
    slope = attrs.get('Slope', 1.0)
    intercept = attrs.get('Intercept', 0.0)
    base = attrs.get('Base', 10.0)
    
    # Apply transformation: base ** (slope * data + intercept)
    # Masking out land/fill values (usually -1 or 255) to avoid math errors
    invalid = (data <= 0) | (data == 255)
    chl = np.full(data.shape, np.nan)
    chl[~invalid] = base ** (slope * data[~invalid] + intercept)
    
    return chl #, proj_info

# Get latitude/longitude grid
hdf_file = "../data/cal_aco_3840_Latitude_Longitude.hdf"
hdf = SD(hdf_file, SDC.READ)
lat_name = list(hdf.datasets().keys())[0]
lon_name = list(hdf.datasets().keys())[1]
lats = hdf.select(lat_name)
lons = hdf.select(lon_name)
lats = lats[:]
lons = lons[:]

# Load chlorophyll file lookup index
with open('../data/chl_index.json', 'r') as f:
    chl_index = json.load(f)
    
date = '2006-07-30' # Using your example file date

if date in chl_index:
    print(chl_index[date])
    chl_data = load_chl_day(chl_index[date])

    assert chl_data.shape == lats.shape == lons.shape

    # Create xarray
    time = np.datetime64(date)
    chl = chl_data[np.newaxis, :, :]
    da = xr.DataArray(
        data=chl,
        dims=("time", "y", "x"),
        coords={
            "time": [time],
            "latitude": (("y", "x"), lats),
            "longitude": (("y", "x"), lons),
        },
        name="CHL",
    )

########## Visualize CHL data for selected day

fig = plt.figure()
stride = 5  # try 8–15 if needed

da_sub = da.isel(y=slice(None, None, stride),x=slice(None, None, stride),)
df = (da_sub.isel(time=0).to_dataframe().reset_index().dropna(subset=["CHL"]))
chl = df["CHL"].values
log_chl = np.log10(chl)
# robust range (avoid outliers)
vmin, vmax = np.nanpercentile(log_chl, [5, 95])
norm = np.clip((log_chl - vmin) / (vmax - vmin), 0, 1)
cmap = plt.get_cmap("viridis")
rgba = cmap(norm)  # values in [0,1]

df["r"] = (rgba[:, 0] * 255).astype(int)
df["g"] = (rgba[:, 1] * 255).astype(int)
df["b"] = (rgba[:, 2] * 255).astype(int)

layer = pdk.Layer(
    "ScatterplotLayer",
    data=df,
    get_position=["longitude", "latitude"],
    get_fill_color="[r, g, b, 300]",
    get_radius=2000,
    pickable=True,
)

view_state = pdk.ViewState(
    latitude=float(df.latitude.mean()),
    longitude=float(df.longitude.mean()),
    zoom=3,
)

deck = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
)

deck

In [ ]:
# Load Integrated NPP Data from CalCOFI Bottle Survey
npp = pd.read_csv('../data/CalCOFI_Integrated_NPP.csv', index_col=0)
npp = npp[['Date', 'Latitude', 'Longitude', 'Integrated_NPP']]
npp['Date'] = pd.to_datetime(npp['Date'])
npp = npp.sort_values(by='Date').reset_index(drop=True)
npp = npp[(npp.Date >= '1998-01-01') & (npp.Date <= '2007-12-31')].reset_index(drop=True)

# Get CHL latitude/longitude grid
hdf_file = "../data/cal_aco_3840_Latitude_Longitude.hdf"
hdf = SD(hdf_file, SDC.READ)
lat_name = list(hdf.datasets().keys())[0]
lon_name = list(hdf.datasets().keys())[1]
lats = hdf.select(lat_name)
lons = hdf.select(lon_name)
lats = lats[:]
lons = lons[:]

tree = cKDTree(np.column_stack([lats.ravel(), lons.ravel()]))

# Query the KDTree for each NPP row
lat_lon = npp[['Latitude', 'Longitude']].values
distances, indices = tree.query(lat_lon)

npp_sat = npp.copy()
npp_sat['chl_sat_idx'] = indices

# Assume da has dims ("time", "y", "x") and chl_sat_idx is in npp_sat
flat_indices = npp_sat['chl_sat_idx'].values

# Convert flat indices to 2D pixel indices (y, x)
y_idx, x_idx = np.unravel_index(flat_indices, da.shape[1:])  # skip time dim

# Store as a tuple array in a new column
npp_sat['chl_sat_grid_idx'] = list(zip(y_idx, x_idx))

# Calcualte grid size and min_valid_pixels for satellite data
res_km = 9
grid_size, min_valid_pixels = calculate_grid_window(res_km)

print(min_valid_pixels)

half = grid_size // 2

# Get the central y/x indices as arrays
y_idx = np.array([y for y, x in npp_sat['chl_sat_grid_idx']])
x_idx = np.array([x for y, x in npp_sat['chl_sat_grid_idx']])

# Compute relative offsets for the grid
offsets = np.arange(-half, half + 1)  # [-1, 0, 1]
dy, dx = np.meshgrid(offsets, offsets, indexing='ij')  # shape (3,3)

# Broadcast to all rows
y_grid = y_idx[:, None, None] + dy[None, :, :]  # shape (n_rows, 3, 3)
x_grid = x_idx[:, None, None] + dx[None, :, :]  # shape (n_rows, 3, 3)

# Clip indices to valid range
y_grid = np.clip(y_grid, 0, da.shape[1] - 1)
x_grid = np.clip(x_grid, 0, da.shape[2] - 1)

# Store as a single array of tuples per row if you want
chl_grid_idxs = np.array(list(zip(y_grid.reshape(len(npp_sat), -1),
                                  x_grid.reshape(len(npp_sat), -1))))
# This gives an array of shape (n_rows, 9, 2), i.e., 3x3 flattened
npp_sat['chl_grid_idxs'] = list(chl_grid_idxs)

display(npp_sat)

display(npp_sat.loc[0,'chl_grid_idxs'])

In [ ]:
import numpy as np
import pandas as pd
import pydeck as pdk

# -----------------------------
# 1. Extract lat/lon for satellite windows
# -----------------------------

def flatten_grid_latlon(row, lats, lons):
    """
    Given a row with chl_grid_idxs (3x3 or grid_size x grid_size), 
    return a DataFrame with columns latitude, longitude, and npp_id.
    """
    grid_idxs = np.array(row['chl_grid_idxs'])  # shape: 2 x 9 (or 9x2?)
    
    # If your chl_grid_idxs is a list of [y_list, x_list] for 3x3
    # convert to numpy array of shape (2, n_pixels)
    if len(grid_idxs) == 2 and len(grid_idxs[0]) == len(grid_idxs[1]):
        y_idx, x_idx = grid_idxs
    else:
        # handle case where it's list of 2D pairs [[y1,x1],...]
        grid_array = np.array(grid_idxs)
        y_idx = grid_array[:,0]
        x_idx = grid_array[:,1]
    
    # extract lat/lon from the original arrays
    lat_vals = lats[y_idx, x_idx]
    lon_vals = lons[y_idx, x_idx]
    
    # return a DataFrame
    df = pd.DataFrame({
        'latitude': lat_vals,
        'longitude': lon_vals,
        'npp_idx': row.name  # link to original NPP point
    })
    return df

# Flatten all rows into one DataFrame for satellite points
sat_points_list = [flatten_grid_latlon(row, lats, lons) for _, row in npp_sat.iterrows()]
sat_points_df = pd.concat(sat_points_list, ignore_index=True)

# -----------------------------
# 2. Prepare NPP points
# -----------------------------
npp_points_df = npp_sat[['Latitude', 'Longitude']].copy()
npp_points_df.rename(columns={'Latitude':'latitude', 'Longitude':'longitude'}, inplace=True)

# -----------------------------
# 3. Create PyDeck layers
# -----------------------------
# NPP points layer (red)
npp_layer = pdk.Layer(
    "ScatterplotLayer",
    data=npp_points_df,
    get_position='[longitude, latitude]',
    get_color='[255, 0, 0]',
    get_radius=200,
    pickable=True,
    auto_highlight=True,
    tooltip=True,
)

# Satellite grid points layer (blue)
sat_layer = pdk.Layer(
    "ScatterplotLayer",
    data=sat_points_df,
    get_position='[longitude, latitude]',
    get_color='[0, 0, 255]',
    get_radius=200,
    pickable=True,
    auto_highlight=True,
)

# -----------------------------
# 4. Create Deck
# -----------------------------
mid_lat = npp_points_df['latitude'].mean()
mid_lon = npp_points_df['longitude'].mean()

r = pdk.Deck(
    layers=[npp_layer, sat_layer],
    initial_view_state=pdk.ViewState(
        latitude=mid_lat,
        longitude=mid_lon,
        zoom=5,
        pitch=0,
    ),
    tooltip={"text": "Latitude: {latitude}\nLongitude: {longitude}"}
)

# Display in Jupyter
r

In [ ]:
def get_chl_window(i, npp_sat, max_day_offset=3, min_valid_pixels=3):
    row = npp_sat.iloc[i]
    date = row['Date']
    grid_idxs = np.array(row['chl_grid_idxs'])
    orig_date = pd.to_datetime(date)

    for offset in range(max_day_offset + 1):
        for direction in [-1, 1] if offset > 0 else [0]:
            current_offset = lag + (direction + offset)
            target_date = orig_date + pd.Timedelta(days=current_offset)
            date_str = target_date.strftime('%Y-%m-%d')
    
            if date.strftime('%Y-%m-%d') in chl_index:
                chl_file = chl_index[target_date.strftime('%Y-%m-%d')]
                chl_data = load_chl_day(chl_file)
            
                # Make xarray
                da = xr.DataArray(
                    data=chl_data[np.newaxis, :, :],  # add time dim
                    dims=("time", "y", "x"),
                    coords={
                        "time": [np.datetime64(target_date)],
                        "latitude": (("y","x"), lats),
                        "longitude": (("y","x"), lons),
                    },
                    name="CHL",
                )
                
                # Extract values for 3x3 grid
                time_idx = 0  # first/only time
                y_idx = grid_idxs[0, :]
                x_idx = grid_idxs[1, :]
            
                chl_window = da.values[time_idx, y_idx, x_idx].reshape(grid_size, grid_size)
                
                # Check for validity
                valid_count = np.isfinite(chl_window).sum()
                if valid_count >= min_valid_pixels:
                    return i, chl_window, current_offset
            
    else:
        return i, None, None

def run_get_chl_window(i, npp_sat, max_day_offset, min_valid_pixels):
    try:
        i, window, offset = get_chl_window(i, npp_sat, max_day_offset, min_valid_pixels)
        return i, window, offset
    except:
        return i, None, None


max_day_offset = 12
lag = 0

var = 'CHL'

col_values = f'{var}_values_lag_{lag}'
col_offset = f'{var}_offset_days_lag_{lag}'
npp_sat[col_values] = None
npp_sat[col_offset] = None


results = Parallel(n_jobs=-1)(
    delayed(run_get_chl_window)(
        i, npp_sat, max_day_offset, min_valid_pixels
    ) for i in range(len(npp))
)

# Add results to NPP dataframe
for result in results:
    if result is not None:
        i, window, offset = result
        
        npp_sat.at[i, col_values] = window
        npp_sat.at[i, col_offset] = offset


# for i in range(len(npp)):

#     out = get_chl_window(i, date, grid_idxs, max_day_offset=3, min_valid_pixels=3)
#     display(out)

In [ ]:
def apply_eppley_model(row):
    """
    Computes NPP using the Eppley square root model for nested array data.
    """
    chl_data = row.get('CHL_values_lag_0')
    
    # Handle None, NaNs, or empty entries
    if chl_data is None or (isinstance(chl_data, float) and np.isnan(chl_data)):
        return pd.Series({'Satellite_NPP (ESQRT)': np.nan, 'Satellite_NPP_std (ESQRT)': np.nan})

    # Convert to numpy array and flatten in case it's a list of lists
    chl_array = np.array(chl_data).flatten()
    
    # Filter for valid, positive chlorophyll values
    valid_chl = chl_array[(chl_array > 0) & np.isfinite(chl_array)]

    if len(valid_chl) == 0:
        return pd.Series({'Satellite_NPP (ESQRT)': np.nan, 'Satellite_NPP_std (ESQRT)': np.nan})

    # The Eppley Square Root Model: NPP = 1000 * sqrt(Chl)
    npp_estimates = 1000 * np.sqrt(valid_chl)

    return pd.Series({
        'Satellite_NPP (ESQRT)': np.mean(npp_estimates),
        'Satellite_NPP_std (ESQRT)': np.std(npp_estimates)
    })

# Apply the function across the rows
eppley_results = npp_sat.apply(apply_eppley_model, axis=1)
npp_eppley = pd.concat([npp, eppley_results], axis=1)
npp_eppley

In [ ]:
# Filter out zero values for log transform

result_df = npp_eppley.copy()
result_df = result_df[result_df['Integrated_NPP'] > 0]

# Create plot
plt_ = make_scatter_plot(result_df, 'ESQRT')
plt_.ylim(10**1.75,10**4)
plt_.xlim(10**1.75,10**4)
plt_.show()

In [ ]:
# def get_sat_values(i, npp, chl_index, grid_size, max_day_offset=3, min_valid_pixels=3):
#     row_npp = npp.loc[i]
#     target_lat, target_lon = row_npp['Latitude'], row_npp['Longitude']
#     orig_date = pd.to_datetime(row_npp['Date'])
    
#     # 1. Load data for the day
#     date_str = orig_date.strftime('%Y-%m-%d')
    
#     # if date_str not in chl_index:
#     #     return i, None, None # Or handle offset search here

#     chl_data, p_info = load_chl_day(chl_index[date_str])
    
#     # 2. Get the exact pixel for this Lat/Lon in Albers space
#     row_idx, col_idx = latlon_to_pixel(target_lat, target_lon, p_info)
    
#     # 3. Extract n x n window. Search day offsets (0, then ±1, ±2, up to max_day_offset)
#     half = grid_size // 2
#     for offset in range(max_day_offset + 1):
#         for direction in [-1, 1] if offset > 0 else [0]:
#             current_offset = lag + (direction * offset)
#             target_date = orig_date + pd.Timedelta(days=current_offset)
#             date_str = target_date.strftime('%Y-%m-%d')

#             # Load the full day's data
#             if date_str not in chl_index:
#                 return i, None, None
                
#             chl_data, p_info = load_chl_day(chl_index[date_str])
            

#             # Get window
#             window = chl_data[row_idx-half : row_idx+half+1, 
#                               col_idx-half : col_idx+half+1]
                
#             # Check for validity
#             valid_count = np.isfinite(window).sum()
#             if valid_count >= min_valid_pixels:
#                 return i, window, current_offset

#     return i, None, None

# def run_get_sat_values(i, npp, chl_index, grid_size, max_day_offset=3, min_valid_pixels=3):
#     try:
#         i, window, offset = get_sat_values(i, npp, chl_index, grid_size, max_day_offset, min_valid_pixels)
#         return i, window, offset
#     except:
#         return i, None, None

# Load Integrated NPP Data from CalCOFI Bottle Survey
npp = pd.read_csv('../data/CalCOFI_Integrated_NPP.csv', index_col=0)
npp = npp[['Date', 'Latitude', 'Longitude', 'Integrated_NPP']]
npp['Date'] = pd.to_datetime(npp['Date'])
npp = npp.sort_values(by='Date').reset_index(drop=True)
npp = npp[(npp.Date >= '1998-01-01') & (npp.Date <= '2007-12-31')].reset_index(drop=True)

# Calcualte grid size and min_valid_pixels for satellite data
res_km = 9
grid_size, min_valid_pixels = calculate_grid_window(res_km)

# Add satellite data to the NPP dataframe
input_vars = ['CHL'] #list(satellite_ds.data_vars)

lags = [0] #[-28, -21, -14, -7, 0, 7, 14, 21, 28]
particle_tracking = False

total_iterations = len(input_vars) * len(lags)
# with tqdm(total=total_iterations) as pbar:
for var in input_vars:
    for lag in lags:
        print(var, lag)
        npp = npp[['Date', 'Latitude', 'Longitude', 'Integrated_NPP']].copy()
        col_values = f'{var}_values_lag_{lag}'
        col_offset = f'{var}_offset_days_lag_{lag}'
        npp[col_values] = None
        npp[col_offset] = None

        results = Parallel(n_jobs=-1)(
            delayed(run_get_sat_values)(
                i, npp, chl_index, grid_size
            ) for i in range(len(npp))
        )
        
        # Add results to NPP dataframe
        for result in results:
            if result is not None:
                i, window, offset = result
                
                npp.at[i, col_values] = window
                npp.at[i, col_offset] = offset

            # # Save individual columns immediately after computation
            # if particle_tracking == True:
            #     save_folder_name = f'npp_columns/{var}_lag_{lag}.pkl'
            # else:
            #     save_folder_name = f'npp_columns_no_particle_tracking/{var}_lag_{lag}.pkl'
            # with open(save_folder_name, 'wb') as f:
            #     pickle.dump({
            #         col_values: npp[col_values],
            #         col_offset: npp[col_offset]
            #     }, f)
                
            # pbar.update(1)

In [ ]:
npp_sat

In [ ]:
npp_eppley = pd.concat([npp, eppley_results], axis=1)
npp_eppley

In [ ]:
# Filter out zero values for log transform

result_df = npp_sat.copy()
result_df = result_df[['Integrated_NPP', 'Satellite_NPP (ESQRT)', 'Satellite_NPP_std (ESQRT)']]
result_df = result_df[result_df['Integrated_NPP'] > 0]

# Create plot
plt = make_scatter_plot(result_df, 'ESQRT')
plt.ylim(10**1.75,10**4)
plt.xlim(10**1.75,10**4)
plt.show()

In [ ]:
def make_scatter_plot(result_df, model_name, ax=None):

    # # Calculate summary statistics (on log-transformed data)
    result_df = result_df[['Integrated_NPP', f'Satellite_NPP ({model_name})', f'Satellite_NPP_std ({model_name})']].copy().dropna()
    log_x = np.log10(result_df['Integrated_NPP'].values)
    log_y = np.log10(result_df[f'Satellite_NPP ({model_name})'].values)

    # Linear regression in log-log space
    model = LinearRegression()
    model.fit(log_x.reshape(-1, 1), log_y)
    log_y_pred = model.predict(log_x.reshape(-1, 1))
    r2 = r2_score(log_y, log_y_pred)
    rmse = np.sqrt(mean_squared_error(log_y, log_y_pred))
    rmsd = np.sqrt(np.mean((log_y - log_y_pred)**2))
    slope = model.coef_[0]
    intercept = model.intercept_
    corr = np.corrcoef(result_df['Integrated_NPP'], result_df[f'Satellite_NPP ({model_name})'])[0,1]
    log_corr = np.corrcoef(log_x, log_y)[0,1]
    n = len(result_df)

    if ax is None:
        fig, ax = plt.subplots(figsize=(6,6))
    
    ax.errorbar(
        result_df['Integrated_NPP'], 
        result_df[f'Satellite_NPP ({model_name})'], 
        yerr=result_df[f'Satellite_NPP_std ({model_name})'], 
        fmt='none',           # no marker (we already have scatterplot)
        ecolor='gray',        # color of error bars
        alpha=0.4,            # transparency
        capsize=2,            # caps on error bars
        elinewidth=1,
        label='±1 Std Dev',
        zorder=1
    )
    
    # Re-transform regression line to original scale
    x_range = np.logspace(np.log10(result_df['Integrated_NPP'].min()), np.log10(result_df['Integrated_NPP'].max()), 100)
    y_fit = 10 ** model.predict(np.log10(x_range).reshape(-1, 1))
    
    if ax is None:
        sns.scatterplot(data=result_df, x='Integrated_NPP', y=f'Satellite_NPP ({model_name})', s=20, alpha=0.8)
    else:
        sns.scatterplot(data=result_df, x='Integrated_NPP', y=f'Satellite_NPP ({model_name})', ax=ax, s=20, alpha=0.8)
        
    # Add regression line
    ax.plot(x_range, y_fit, color='red', label='Regression line')
    
    # Add 1:1, 1:2, and 1:3 lines
    min_val = min(result_df[['Integrated_NPP', f'Satellite_NPP ({model_name})']].min())
    max_val = max(result_df[['Integrated_NPP', f'Satellite_NPP ({model_name})']].max())
    ax.plot(x_range, x_range, 'k--', label='1:1 line', alpha=0.5)
    ax.plot(x_range, x_range / 2, 'k:', label='1:2 line', alpha=0.5)
    ax.plot(x_range, x_range / 3, 'k-.', label='1:3 line', alpha=0.5)
    
    # Labels, scales, title
    ax.set_xlabel('In situ NPP (mg C/m²/day)')
    ax.set_ylabel(f'Satellite NPP ({model_name})')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(f'Satellite vs. In Situ NPP ({model_name})')

    # Add R², RMSE, N as text
    ax.text(0.05, 0.95, 
            f'$R^2$ = {r2:.2f}\n'
            f'RMSE = {rmse:.2f}\n'
            f'RMSD = {rmsd:.2f}\n'
            f'Slope = {slope:.3f}\nIntercept = {intercept:.3f}\n'
            f'ρ = {corr:.3f}\n'
            f'ρ (log-log) = {log_corr:.3f}\n'
            f'N = {n}',
            transform=ax.transAxes, 
            verticalalignment='top',
            fontsize=9,
            bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))
    
    ax.grid(True, which='both', ls=':')
    ax.legend(loc='lower right')

    return ax

In [ ]:


def get_day_length(row):
    try:
        date = pd.to_datetime(row['Date']).date()
        observer = Observer(latitude=row['Latitude'], longitude=row['Longitude'])
        s = sun(observer, date=date)

        # Convert UTC times to local timezone (e.g., US/Pacific)
        local_tz = pytz.timezone('US/Pacific')

        sunrise_local = s['sunrise'].astimezone(local_tz)
        sunset_local = s['sunset'].astimezone(local_tz)

        if sunset_local < sunrise_local:
            sunset_local += timedelta(days=1)

        # Calculate day length in hours
        day_length_hours = (sunset_local - sunrise_local).total_seconds() / 3600

        return day_length_hours
        
    except Exception as e:
        return None
